In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('../data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 0. Товар 999999

На вебинаре мы использовали товар 999999 - что это за товар?  
    
    Искуственный товар который присвоен всем пользователям не вошедших в "топ".

Зачем он нужен?  
    
    Для того что бы не обрезать выборку

Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
    
        Улучшаем качество за счет не уменьшения выборки

In [39]:
data_train_webinar = pd.read_csv('./data_train.csv')
data_test_webinar = pd.read_csv('./data_test.csv')

In [40]:
data_train_webinar.loc[data_train_webinar['item_id']== 999999]

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
6,2375,26984851516,1,999999,1,1.57,364,-0.68,1642,1,0.0,0.0
8,2375,26984851516,1,999999,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,999999,1,2.00,364,-0.79,1642,1,0.0,0.0
10,2375,26984851516,1,999999,1,2.00,364,-0.79,1642,1,0.0,0.0
12,1364,26984896261,1,999999,1,2.99,31742,-0.40,1520,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2278476,2088,41297771158,635,999999,1,15.65,304,-5.25,1258,91,0.0,0.0
2278479,1541,41297771177,635,999999,1,1.05,304,0.00,1300,91,0.0,0.0
2278480,1541,41297771180,635,999999,1,5.19,304,0.00,1301,91,0.0,0.0
2278481,1168,41297772063,635,999999,1,12.40,304,0.00,1526,91,0.0,0.0


Можно ли удалить этот товар?   
    
    Можно но ухудшим качество

Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

In [43]:
data_train_webinar = data_train_webinar.drop(data_train_webinar[data_train_webinar.item_id == 999999].index)

In [44]:
data_train_webinar.loc[data_train_webinar['item_id']== 999999]

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [4]:
items_weights_1 = data.groupby('item_id')['sales_value'].sum().reset_index().rename({'sales_value': 'weight'}, axis=1) 
const_weights = 2    # что бы избавиться от 0 
items_weights_1['weight'] = np.log(items_weights_1['weight'] + const_weights)
items_weights_1['weight'] = items_weights_1['weight']/items_weights_1['weight'].sum()

In [5]:
items_weights_1.describe()

,item_id,weight
count,8.905100e+04,89051.000000
mean,5.115772e+06,0.000011
std,5.178973e+06,0.000006
min,2.567100e+04,0.000003
25%,9.665830e+05,0.000007
50%,1.448516e+06,0.000010
75%,9.553042e+06,0.000015
max,1.802456e+07,0.000050


In [6]:
items_weights_1.sum()

item_id    4.555646e+11
weight     1.000000e+00
dtype: float64

In [7]:
items_weights_2 = data.groupby('item_id')['sales_value'].sum().reset_index().rename({'sales_value': 'weight'}, axis=1) 

items_weights_2['weight'] = items_weights_2['weight'] + const_weights
items_weights_2['weight'] = items_weights_2['weight']/items_weights_2['weight'].sum()

In [8]:
items_weights_2.sum()

item_id    4.555646e+11
weight     1.000000e+00
dtype: float64

In [9]:
items_weights_3 = data.groupby('item_id')['sales_value'].sum().reset_index().rename({'sales_value': 'weight'}, axis=1) 

items_weights_3['weight'] = const_weights - 1/np.e**(items_weights_3['weight'])
items_weights_3['weight'] = items_weights_3['weight']/items_weights_3['weight'].sum()

In [10]:
items_weights_3.sum()

item_id    4.555646e+11
weight     1.000000e+00
dtype: float64

In [11]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    recs = np.random.choice(items_weights['item_id'], p=items_weights['weight'],  size=n, replace=False)
    
    
    return recs.tolist()

Сделайте предсказания

In [12]:
result1 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result1.columns=['user_id', 'actual']
result1['actual'] = result1['actual'].apply(lambda x: list(x))
result1.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [13]:
%%time
result1['weighted_random_recommendation'] = result1['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_1))
result1.head(10)
# your_code

CPU times: user 1.79 s, sys: 4.12 ms, total: 1.79 s
Wall time: 1.79 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[124053, 13072779, 1095350, 1113285, 6979484]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[843819, 875686, 12333992, 952893, 121043]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[15452479, 1018977, 984957, 13186980, 833919]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[920833, 6961375, 6442453, 819933, 1102057]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[6534218, 1095394, 9677255, 957063, 9245461]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[819809, 65291, 27334, 10145318, 1112828]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[1105750, 8019844, 980666, 9680965, 983124]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[2379378, 7144468, 1130594, 1087581, 7167903]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[908883, 12604515, 1036232, 1054991, 2549285]"
9,16,"[1062973, 1082185, 13007710]","[1401480, 935516, 7466828, 5565392, 963727]"


In [14]:
result2 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result2.columns=['user_id', 'actual']
result2['actual'] = result2['actual'].apply(lambda x: list(x))

In [15]:
%%time
result2['weighted_random_recommendation'] = result2['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_2))
result2.head(10)

CPU times: user 1.83 s, sys: 255 µs, total: 1.83 s
Wall time: 1.83 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[7147915, 879977, 970952, 970792, 851066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1070782, 995660, 892156, 825882, 1013895]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1096792, 8249063, 869177, 831587, 1127179]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[7151074, 1087362, 883616, 986417, 1082185]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[13381584, 936830, 869054, 6534178, 9297602]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[1043590, 1038901, 6534178, 1047106, 9527159]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[6513723, 860975, 1005186, 12524691, 951197]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[6919355, 15775751, 826606, 860037, 893810]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[899115, 6534178, 6704329, 844179, 12384775]"
9,16,"[1062973, 1082185, 13007710]","[6034577, 1029743, 853178, 1060067, 986947]"


In [16]:
result3 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result3.columns=['user_id', 'actual']
result3['actual'] = result3['actual'].apply(lambda x: list(x))

In [17]:
%%time
result3['weighted_random_recommendation'] = result3['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_3))
result3.head(10)

CPU times: user 2.01 s, sys: 20.1 ms, total: 2.03 s
Wall time: 2.03 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[396311, 929018, 801072, 1423201, 99875]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[872453, 5565363, 932428, 12132026, 14077362]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[737973, 881763, 98398, 7167350, 9522352]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[917254, 5568214, 1254671, 2052742, 952657]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1818418, 1010727, 1599432, 1113472, 864346]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[9297058, 15595751, 1070926, 960075, 892502]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[999074, 8293365, 977884, 13073055, 942868]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[13986496, 6396570, 7166724, 1413717, 6445236]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[1053595, 13381467, 1025346, 7169122, 12188511]"
9,16,"[1062973, 1082185, 13007710]","[13877305, 961057, 12758851, 354984, 985000]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. 

In [18]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [19]:
result = pd.read_csv('preds.csv')# закгрузка predict с семианара
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9553121, 864744, 5722204, 649483, 2496427]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12949875, 555470, 914974, 12263714, 882047]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [20]:
type(result['actual'][0])

str

In [21]:
# файл с предсказаниями сохранил предсказания как строки
# нужно перевести обратно в список
for a in result.columns[1:]:
    result[a]=  result[a].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))

In [32]:
random_recommendation_result = result.apply(
    lambda x: precision_at_k(x['random_recommendation'], x['actual'],  5), axis=1
    ).mean()

popular_recommendation_result = result.apply(
    lambda x: precision_at_k(x['popular_recommendation'], x['actual'],  5), axis=1
    ).mean()
    
itemitem_result = result.apply(
    lambda x: precision_at_k(x['itemitem'], x['actual'],  5), axis=1
    ).mean()
    
cosine_result = result.apply(
    lambda x: precision_at_k(x['cosine'], x['actual'],  5), axis=1
    ).mean()
    
tfidf_result = result.apply(
    lambda x: precision_at_k(x['tfidf'], x['actual'],  5), axis=1
    ).mean()
    
own_purchases_result = result.apply(
    lambda x: precision_at_k(x['own_purchases'], x['actual'],  5), axis=1
    ).mean()

    
weighted_random_recommendation_result1 = result1.apply(
    lambda x: precision_at_k(x['weighted_random_recommendation'], x['actual'],  5), axis=1
    ).mean()
    
weighted_random_recommendation_result2 = result2.apply(
    lambda x: precision_at_k(x['weighted_random_recommendation'], x['actual'],  5), axis=1
    ).mean()
    
weighted_random_recommendation_result3 = result3.apply(
    lambda x: precision_at_k(x['weighted_random_recommendation'], x['actual'],  5), axis=1
    ).mean()
    
result_recom_dict = {
    'random_recommendation_result':random_recommendation_result,
    'weighted_random_recommendation_result1': weighted_random_recommendation_result1,
    'weighted_random_recommendation_result2': weighted_random_recommendation_result2,
    'weighted_random_recommendation_result3': weighted_random_recommendation_result3,
    'popular_recommendation_result': popular_recommendation_result,
    'itemitem_result(k=5)': itemitem_result,
    'cosine_result': cosine_result,
    'tfidf_result': tfidf_result,
    'own_purchases_result': own_purchases_result,
    }

In [35]:
for key, value in result_recom_dict.items(): print(f'{key} \t\t {value}') 

random_recommendation_result 		 0.00019588638589618023
weighted_random_recommendation_result1 		 0.0005876591576885406
weighted_random_recommendation_result2 		 0.021449559255631762
weighted_random_recommendation_result3 		 0.00039177277179236047
popular_recommendation_result 		 0.15523996082272082
itemitem_result(k=5) 		 0.13692458374142857
cosine_result 		 0.13290891283055686
tfidf_result 		 0.1389813907933383
own_purchases_result 		 0.17969311132876015


Какой алгоритм показывает лучшее качество?

    own_purchases(трюк с itemitem для ближашего 1-ого соседа)
    
Почему?
    
    рекомендация на основе наипохожего товара

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [24]:
# your_code

### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.